In [1]:
# improts for our used libs
import utils
import torch
import pandas as pd
import numpy as np
import model_simple as ms

from torch import nn
from data_loader import get_loader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

## Create X for model and y for decision tree

In [ ]:
timeframe = 30

In [ ]:
utils.create_X_y('PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', loss_ratio=3, timeframe=30)

In [ ]:
# now we want to create our csv files for X and for y (X = the input for our xgboost and y the output of our xgboost)
# remeber X will be also used for our Autoencoders
X = pd.read_csv('data/X_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv')
y = pd.read_csv('data/y_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv')

### Just for evaulation

In [ ]:
# check if same length in the table
print(X.shape[0] == y.shape[0])
# show me which values we got :-)
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

## create model

In [ ]:
# timeframe is meant the shape of tensor regarding fields
# so can be diffrent from timeframe abofe
ts = 60*24
data = get_loader(data_path='data/X_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', timeframe=ts, batch_size=128)
# set learning rate and input dim
lr = 1e-1
ds = iter(data)
input_dim = ds.next().shape[2]
# inizialise model
model = ms.SimpleAutoencoder(input_dim=input_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# model # to print my model

In [ ]:
# train model
epoch_num = 1
model.train()
for epoch in range(epoch_num):
    for i, x in enumerate(data):
        # forward
        x = x.float()
        y = model(x).float()
        loss = criterion(y, x)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # ------- log output ---------
    print('epoch [{}/{}], loss:{:.6f}, MSE_loss:{:.6f}'
          .format(epoch + 1, epoch_num, loss.data, 0))

## evaluate and save model result

In [ ]:
dp = get_loader(data_path='data/X_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', timeframe=ts, batch_size=1)
# prefill results until first dataloader element
result = [0] * ts
for i, x in enumerate(dp):
    # add value to pandas 
    result.append(criterion(x.float(), model(x.float())).data.numpy())
pd.DataFrame(result).to_csv('data/ym__PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', header=None, index=None)

## Decision tree

In [2]:
X = pd.read_csv('data/ym__PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', header=None)
y = y = pd.read_csv('data/y_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv')

In [3]:
# replace 1 and 2 with just 1
y.replace(to_replace=[2], value=1, inplace=True)

In [4]:
X_train = pd.DataFrame(X[0])[300000:].reset_index(drop=True)
y_train = y['2'].values.reshape(1,-1)[0][300000:]

X_test = pd.DataFrame((X[0])[:300000]).reset_index(drop=True)
y_test = y['2'].values.reshape(1,-1)[0][:300000]

In [ ]:
# initialise randomforest
clf = RandomForestClassifier()
# fit 
clf.fit(X_train, y_train)
# predict results
pred = clf.predict(X_test)

In [ ]:
# print confusion matrix # 600 estimators
conf = confusion_matrix(y_test, pred)
print (conf)

In [ ]:
# print confusion matrix # 10 estimators
conf = confusion_matrix(y_test, pred)
print (conf)

In [ ]:
# diffrent confutionmatrix for diffrent time rames and also diffrence in %

In [ ]:
# get dates and time for the true positve to cross check

In [ ]:
X_train_df = pd.DataFrame()
X_test_df = pd.DataFrame()

for i in range(20):
    pct_change = i * 5    
    if(pct_change == 0):
        pct_change = 1
    print(pct_change)
    X_train_ = X_train.pct_change(periods=pct_change)
    X_test_ = X_test.pct_change(periods=pct_change)
    X_test_.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_train_.fillna(0, inplace=True)
    X_test_.fillna(0, inplace=True)
    X_train_ = pd.DataFrame(X_train_, dtype='float32')
    X_test_ = pd.DataFrame(X_test_, dtype='float32')
    # initialise randomforest
    clf = RandomForestClassifier(n_estimators=100)
    # fit 
    clf.fit(X_train_, y_train)
    pred = clf.predict(X_test_)
    # conf matrix
    conf = confusion_matrix(y_test, pred)
    print (conf)
    
    
    X_train_ = X_train.pct_change(periods=pct_change)
    X_test_ = X_test.pct_change(periods=pct_change)
    X_test_.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_train_.fillna(0, inplace=True)
    X_test_.fillna(0, inplace=True)
    X_train_ = pd.DataFrame(X_train_, dtype='float32')
    X_test_ = pd.DataFrame(X_test_, dtype='float32')
    # initialise randomforest
#     X_train_.values.reshape(-1,1)
    X_train_df[str(pct_change)] = X_train_.values.reshape(-1,)
    X_test_df[str(pct_change)] = X_test_.values.reshape(-1,)
    
    # fit 
    if i > 0:
        y_train_ = np.insert(y_train, 0, ([0 for p in range(i)]))[:-i]
        y_test_ = np.insert(y_test, 0, ([0 for p in range(i)]))[:-i]
        clf.fit(X_train_df, y_train_)
        pred = clf.predict(X_test_df)
        # conf matrix
        print("conf for tf " + str(pct_change))
        conf = confusion_matrix(y_test_, pred)
        print (conf)
    
    # safe conv matrix

1
[[270757   1186]
 [ 27789    268]]
5
[[270682   1261]
 [ 27649    408]]


In [ ]:
X_test_.describe()

In [ ]:
X_train_[X_train_[0] == '-inf']

In [ ]:
X_test_.info(verbose=True)

In [ ]:
pd.DataFrame(y_test).info()

In [2]:
a = np.array([1,2,3,4])
a

array([1, 2, 3, 4])

In [15]:
a = np.insert(y_train, 0, (11, 11, 12))

In [16]:
a[:-1]

array([11, 11, 12, ...,  0,  0,  0])

In [ ]:
p = 1

np.insert(y_train, 0, ([0 for i in range(1)]))[:-p]

In [ ]:
y_train

In [34]:
y_test_ = np.insert(y_test, 0, ([0 for i in range(i)]))[:-i]

In [36]:
X_train_df.shape

(1022728, 1)

In [37]:
y_train_.shape

(0,)

In [ ]:
1
[[270787   1156]
 [ 27798    259]]
5
[[270667   1276]
 [ 27641    416]]
conf for tf 5
[[271324    620]
 [ 27919    137]]
10
[[270295   1648]
 [ 27455    602]]
conf for tf 10
[[271891     54]
 [ 28032     23]]
15
[[269647   2296]
 [ 27316    741]]
conf for tf 15
[[271912     34]
 [ 28030     24]]
20
[[268983   2960]
 [ 27211    846]]
conf for tf 20
[[271917     30]
 [ 28019     34]]
25
[[268549   3394]
 [ 27063    994]]
conf for tf 25
[[271914     34]
 [ 28009     43]]
30
[[267850   4093]
 [ 26962   1095]]
conf for tf 30
[[271912     37]
 [ 27993     58]]
35
[[267397   4546]
 [ 26925   1132]]
conf for tf 35
[[271914     36]
 [ 27971     79]]
40
[[266805   5138]
 [ 26741   1316]]
conf for tf 40
[[271899     52]
 [ 27955     94]]
45
[[266349   5594]
 [ 26650   1407]]
conf for tf 45
[[271881     71]
 [ 27939    109]]
50
[[266216   5727]
 [ 26711   1346]]
conf for tf 50
[[271885     68]
 [ 27915    132]]
55
[[265512   6431]
 [ 26518   1539]]
conf for tf 55
[[271884     70]
 [ 27889    157]]
60
[[265067   6876]
 [ 26448   1609]]
conf for tf 60
[[271888     67]
 [ 27887    158]]
65
[[264961   6982]
 [ 26424   1633]]
conf for tf 65
[[271877     78]
 [ 27862    183]]
70
[[264907   7036]
 [ 26357   1700]]
conf for tf 70
[[271872     84]
 [ 27848    196]]
75
[[264547   7396]
 [ 26362   1695]]
conf for tf 75
[[271879     78]
 [ 27804    239]]
80
[[264352   7591]
 [ 26255   1802]]
conf for tf 80
[[271843    115]
 [ 27777    265]]
85
[[264094   7849]
 [ 26150   1907]]
conf for tf 85
[[271853    106]
 [ 27751    290]]
90
[[263986   7957]
 [ 26266   1791]]
conf for tf 90
[[271839    121]
 [ 27714    326]]
95
[[263633   8310]
 [ 26186   1871]]
conf for tf 95
[[271833    128]
 [ 27684    355]]